從《Python 網路爬蟲與資料分析入門實戰》第三章的範例中練習爬蟲

此為第三章中批踢踢八卦版爬蟲範例中練習自寫程式碼

書中原始程式碼來源：https://github.com/jwlin/web-crawler-tutorial/tree/master/ch3

# 壹、寫爬蟲函數：(1)抓取今日的文章內容(2)重複抓取動作並儲存

In [1]:
import requests
import time
from bs4 import BeautifulSoup

def Gossiping_today_pop_articles(url):
    #1.與網頁溝通/連線(使用requests套件)並帶入是否滿18歲的cookies
    resp=requests.get(url,cookies={'over18': '1'})
    
    #2.剖析網頁原始碼(使用BeautifulSoup剖析器)  
    soup=BeautifulSoup(resp.text,'html5lib')
    
    
    #3.取得該網頁的內容
        #3.1取得上一頁連結：(1)先定位上一頁連結的「大概位置」於標籤名稱div/屬性名稱btn-group btn-group-paging
    paging_div = soup.find('div', 'btn-group btn-group-paging')
        
                        ##(2)再定位「細部精確」的所在位置：找到所有a標籤中第2個a的href(網址)
    PTT_URL = 'https://www.ptt.cc'        
    prev_url = PTT_URL+paging_div.find_all('a')[1]['href']
        
        #3.2取得該網頁的文章內容：(1)設置儲存文章的變數
    articles = []
        
                            ##(2)定位文章內容的區塊：標籤名稱div/屬性名稱r-ent
    divs = soup.find_all('div', 'r-ent')
        
                            ##(3)創造一個今日的日期並符合PTT日期格式
    today = time.strftime("%m/%d").lstrip('0')

                            ##(4)使用for迴圈取得文章區塊內的所需內容
    for d in divs:
        
                            ##(5)先設置條件取得符合今日的文章：如果發文日期(位置：div標籤/date屬性)是今天
        if d.find('div', 'date').text.strip()==today:  
            
                            ##(6)取得推文數並編輯：(6.1)先設置一個為0的推文數
            push_count = 0
        
                                                ##(6.2)取得字串型態的原始推文數:位置div標籤名/nrec屬性
            push_str = d.find('div', 'nrec').text
            
                                                ##(6.3)如果原始推文存在,將字串轉為數字
            if push_str:
                try:
                    push_count = int(push_str) 
                
                                                ##如果轉換失敗表示原始推文數為文字或空白('爆'或 'X1','X2')
                                                ##空白的推文數保持為0
                except ValueError:
            
                                                ##(6.4)轉換失敗的「爆」文轉成99推文數
                    if push_str == '爆':
                        push_count = 99
                        
                                                ##(6.5)轉換失敗的「X」文轉成-10的推文數
                    elif push_str.startswith('X'):
                        push_count = -10
                        
                        
                            ##(7)取得文章連結/標題/作者並存至文章的變數中
                                    ##(7.1)如果文章存在：標籤a就是文章所在位置
            if d.find('a'):
                
                                    ##(7.2)取得文章標題：標籤a的字串
                title = d.find('a').text    
                
                                    ##(7.3)取得文章日期：div標籤/date屬性
                date= d.find('div', 'date').text
                
                                    ##(7.4)取得文章作者：標籤div屬性author的字串
                author = d.find('div', 'author').text 
                
                                    ##(7.5)取得文章網址：標籤a的href
                href = PTT_URL+d.find('a')['href']   
                
                                    ##(7.5)將上述變數儲存至文章的變數中
                articles.append({
                    'title':title,
                    'date':date,
                    'author':author,
                    'push_count':push_count,
                    'href':href})
                
                        ##(8)將程式結果返回文章內容
    return articles,prev_url


#4.取得該網頁的上一頁的文章內容(重複1.2.3的動作)
def repeat_articles(url):
    
    ##(4.1)創造放文章的變數
    articles=[]
    
    ##(4.2)利用自寫函數取得「原網頁文章內容」與「上一頁連結」
    new_articles,prev_url=Gossiping_today_pop_articles(url)
    
    ##(4.3)將原網頁內容放入變數中
    articles += new_articles
    
    ##(4.4)利用字寫函數取得的「上一頁連結」來取得「上一頁的文章內容與連結」
    new_articles, new_prev_url= Gossiping_today_pop_articles(prev_url)
    
    ##(4.5)如果「上一頁內容」不是空的=(文章日期為今日)
        ###因Gossiping_today_pop_articles函數中設定只抓取今日文章,若非今日變數則為[]
    while new_articles!=[]:
        
        ##(4.5.1)將「上一頁文章」放入變數中
        articles += new_articles
        
        ##(4.5.2)創造更新的「上一頁文章與連結」
        new_articles,new_prev_url=Gossiping_today_pop_articles(new_prev_url)
        
    ##(4.6)如果新的「上一頁內容」是空的=(文章日期非今日)則印出上述合併的所有文章內容
    else:
        return articles

# 貳、處理資料：(1)印出>50的今日文章 (2)所有今日文章存成csv檔案

In [2]:
#1.印出今日推文數>50的文章
    ##1.1將自寫函數設為文章變數
url='https://www.ptt.cc/bbs/Gossiping/index.html'
articles=repeat_articles(url)

    ##1.2印出總文章數
print('今天有', len(articles), '篇文章')

    ##1.3印出推文數>50的文章
print('熱門文章(> 50 推):' )    
for a in articles:
    if int(a['push_count']) > 50:
        print(a)

今天有 918 篇文章
熱門文章(> 50 推):
{'title': '[新聞] 王立強共諜案 朱立倫四個字形容', 'date': '11/25', 'author': 'nakibasara', 'push_count': 57, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1574665317.A.F11.html'}
{'title': '[爆卦] 何君堯住院時合照議員全數落選!爽!', 'date': '11/25', 'author': 'Areso', 'push_count': 99, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1574664254.A.4AC.html'}
{'title': '[新聞] 蔡英文下令 國安局抵澳查王立強案', 'date': '11/25', 'author': 'makoto1982', 'push_count': 99, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1574663213.A.F38.html'}
{'title': '[爆卦] 中國人推人臉辨識投票，加快開票', 'date': '11/25', 'author': 'ben1013', 'push_count': 55, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1574662308.A.9CE.html'}
{'title': '[爆卦] 向心在出境台灣時被要求配合調查', 'date': '11/25', 'author': 'amos6064', 'push_count': 99, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1574661568.A.DE1.html'}
{'title': '[新聞] 又一批中國政府秘密新疆文件洩露海外', 'date': '11/25', 'author': 'scarfman', 'push_count': 57, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1574661719.A.46E.html'}
{'title': '[問卦]

In [3]:
import pandas as pd
import numpy as np

#2.儲存所有今日文章
    ##2.1爬取文章
data=repeat_articles(url)

    ##2.2將資料轉成DataFrame型式(columns可不寫入，資料中列的名稱會照字母排序)
name=['title','date','author','push_count','href']
test=pd.DataFrame(columns=name,data=data)

    ##2.3把索引改成從1開始
test.index = np.arange(1,len(test)+1)
    
    ##2.4將索引名稱改為編號
test.index.names = ['article_NO.']

    ##印出結果並儲存成CSV檔案
print(test)
test.to_csv('testcsv.csv',encoding='utf_8_sig')

                                       title   date        author  push_count  \
article_NO.                                                                     
1                 Re: [問卦] 認真在看經濟學人雜誌的都是什麼人？  11/25    qqmaybenot           0   
2                            [問卦] 到菲律賓找老婆如何？  11/25      rockon15          11   
3                   Re: [問卦] 日本人出國比例好像沒台灣人高？  11/25      st900278           7   
4              Re: [新聞] 韓國瑜口誤有「200個大使館」　網酸：把  11/25   ilovesumika           1   
5                Re: [新聞] 獨家訪問／前軍情局副局長：王立強從頭  11/25        kimjay           1   
6                  Re: [爆卦] 向心剛在出境台灣時被要求配合調查  11/25         kh749           5   
7                 Re: [爆卦] 何君堯住院時合照議員全數落選!爽!  11/25       MJdavid           2   
8                Re: [新聞] 面對共諜案 柯文哲批每天喊賊來了沒用  11/25        X18999           5   
9                         [問卦] 11月了還這麼熱是在哈囉？  11/25      gustavvv           0   
10                  [新聞] 機場攔截向心夫婦 調查局：事涉國安，無  11/25     OmegaWind          48   
11               [新聞] 館長陸戰隊時

# 無註解程式碼

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np



def Gossiping_today_pop_articles(url):
    resp=requests.get(url,cookies={'over18': '1'})
    soup=BeautifulSoup(resp.text,'html5lib')
    
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    PTT_URL = 'https://www.ptt.cc'        
    prev_url = PTT_URL+paging_div.find_all('a')[1]['href']
   

    articles = []
    divs = soup.find_all('div', 'r-ent')

    today = time.strftime("%m/%d").lstrip('0')

    
    for d in divs:
        if d.find('div', 'date').text.strip()==today:  
            push_count = 0
            push_str = d.find('div', 'nrec').text
            
            if push_str:
                try:
                    push_count = int(push_str) 
                except ValueError:
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10

            if d.find('a'):
                title = d.find('a').text    
                date= d.find('div', 'date').text
                author = d.find('div', 'author').text 
                href = PTT_URL+d.find('a')['href']   
                articles.append({
                    'title':title,
                    'date':date,
                    'author':author,
                    'push_count':push_count,
                    'href':href})

    return articles,prev_url



def repeat_articles(url):
    articles=[]
    
    new_articles,prev_url=Gossiping_today_pop_articles(url)
    articles += new_articles
    new_articles, new_prev_url= Gossiping_today_pop_articles(prev_url)

    while new_articles!=[]:
        articles += new_articles
        new_articles,new_prev_url=Gossiping_today_pop_articles(new_prev_url)

    else:
        return articles

    
    
    
    
    
    
url='https://www.ptt.cc/bbs/Gossiping/index.html'    
articles=repeat_articles(url)

print('今天有', len(articles), '篇文章')
print('熱門文章(> 50 推):' )    
for a in articles:
    if int(a['push_count']) > 50:
        print(a)


        
        
data=repeat_articles(url)
name=['title','date','author','push_count','href']
test=pd.DataFrame(columns=name,data=data)
test.index = np.arange(1,len(test)+1)
test.index.names = ['article_NO.']


print(test)
test.to_csv('testcsv.csv',encoding='utf_8_sig')

# urllib套件版本的爬蟲

In [ ]:
import urllib.request
import urllib.error
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#urllib.request.urlopen爬取大量資料容易WinError 10054,因此增加關閉resp與設置等待時間

def Gossiping_today_pop_articles(url):
    
    request = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0','cookie': 'over18=1'})
    try:
        resp = urllib.request.urlopen(url = request)
        soup=BeautifulSoup(resp.read(),'html5lib')
    
        paging_div = soup.find('div', 'btn-group btn-group-paging')
        PTT_URL = 'https://www.ptt.cc'        
        prev_url = PTT_URL+paging_div.find_all('a')[1]['href']
   

        articles = []
        divs = soup.find_all('div', 'r-ent')

        today = time.strftime("%m/%d").lstrip('0')

    
        for d in divs:
            if d.find('div', 'date').text.strip()==today:  
                push_count = 0
                push_str = d.find('div', 'nrec').text
            
                if push_str:
                    try:
                        push_count = int(push_str) 
                    except ValueError:
                        if push_str == '爆':
                            push_count = 99
                        elif push_str.startswith('X'):
                            push_count = -10

                if d.find('a'):
                    title = d.find('a').text    
                    date= d.find('div', 'date').text
                    author = d.find('div', 'author').text 
                    href = PTT_URL+d.find('a')['href']   
                    articles.append({
                    'title':title,
                    'date':date,
                    'author':author,
                    'push_count':push_count,
                    'href':href})

        resp.close()
        
    except urllib.error.URLError as e:
        print(e.reason)
        time.sleep(1)
        
    return articles,prev_url
        


def repeat_articles(url):
    articles=[]
    
    new_articles,prev_url=Gossiping_today_pop_articles(url)
    articles += new_articles
    new_articles, new_prev_url= Gossiping_today_pop_articles(prev_url)

    while new_articles!=[]:
        articles += new_articles
        new_articles,new_prev_url=Gossiping_today_pop_articles(new_prev_url)

    else:
        return articles

    
    
    
    
    
    
url='https://www.ptt.cc/bbs/Gossiping/index.html'    
articles=repeat_articles(url)

print('今天有', len(articles), '篇文章')
print('熱門文章(> 50 推):' )    
for a in articles:
    if int(a['push_count']) > 50:
        print(a)


        
        
data=repeat_articles(url)
name=['title','date','author','push_count','href']
test=pd.DataFrame(columns=name,data=data)
test.index = np.arange(1,len(test)+1)
test.index.names = ['article_NO.']


print(test)
test.to_csv('testcsv.csv',encoding='utf_8_sig')